In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, ReLU, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical


In [4]:
import cv2
import numpy as np

In [5]:
from tensorflow.keras import backend as K


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [97]:
SIZE = 256
N = 1000
epochs = 3

In [7]:
# X = np.zeros((N, SIZE, SIZE))
# A = np.zeros((N, SIZE, SIZE))
# B = np.zeros((N, SIZE, SIZE))
# for i in range(1, N + 1):
#     X[i - 1] = cv2.imread('data/bnw/' + str(i + 7) + '.jpeg', 0)
#     A[i - 1] = cv2.imread('data/a/' + str(i + 7) + '.jpeg', 0)
#     B[i - 1] = cv2.imread('data/b/' + str(i + 7) + '.jpeg', 0)



In [8]:
# Data loader
class LoadData(Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, img_paths, a_paths, bins):
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_paths = img_paths
        self.a_paths = a_paths
        self.bins = bins
        self.classes = self.bins.shape[0] - 1

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.img_paths[i: i + self.batch_size]
        batch_a_paths = self.a_paths[i: i + self.batch_size]
        x = np.zeros((self.batch_size, self.img_size, self.img_size), dtype="float32")

        for j, path in enumerate(batch_input_img_paths):
            # img = load_img(path, color_mode='grayscale')
            x[j] = cv2.imread(path, 0)

        a = np.zeros((self.batch_size, self.img_size, self.img_size), dtype="uint8")
        for j, path in enumerate(batch_a_paths):
            # img = load_img(path, color_mode="grayscale")
            a[j] = cv2.imread(path, 0)

        x = x / 255
        a_binned = np.digitize(a, self.bins) - 1
        a_hot = to_categorical(a_binned, num_classes=self.classes)
        x = x.reshape(self.batch_size, self.img_size, self.img_size, 1)
        # output_shape = (self.batch_size, self.img_size, self.img_size, 1)

        return x, a_hot

In [139]:
bins_list = [0, 64, 96, 112, 120, 128, 136, 144, 160, 192, 256]
bin_val = np.array([32, 80, 104, 116, 124, 132, 140, 152, 176, 224])
num_bins = len(bins_list) - 1
bins = np.array(bins_list)
# a_binned = np.digitize(A, bins) - 1
# b_binned = np.digitize(B, bins) - 1

In [10]:
# One hot encode A and B

# a_hot = to_categorical(a_binned)
# b_hot = to_categorical(b_binned)

In [11]:
# X = X.reshape(N, SIZE, SIZE, 1)
# y = np.stack((A, B)).transpose((1, 2, 3, 0))

In [12]:
# Normalize the data
# X = X / 255
# y = y / 255

Following are utility functions for creating a U-net Model

In [13]:
def conv_stack(input_layer, filters):
    conv1 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(input_layer)
    batch_norm1 = BatchNormalization()(conv1)
    relu1 = ReLU()(batch_norm1)

    conv2 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(relu1)
    batch_norm2 = BatchNormalization()(conv2)
    relu2 = ReLU()(batch_norm2)

    return relu2

In [14]:
def encoder_block(input_layer, filters):
    conv = conv_stack(input_layer, filters)
    max_pool = MaxPooling2D(pool_size=(2, 2))(conv)

    return conv, max_pool

In [15]:
def decoder_block(input_layer, skip_layer, filters):
    up = Conv2DTranspose(filters, 2, strides=2, padding='same')(input_layer)
    conc = Concatenate()([up, skip_layer])
    dec = conv_stack(conc, filters)

    return dec

In [16]:
def get_model(size, init_filters):
    inputs = Input((size, size, 1))

    conv1, max_pool1 = encoder_block(inputs, init_filters)
    conv2, max_pool2 = encoder_block(max_pool1, init_filters * 2)
    conv3, max_pool3 = encoder_block(max_pool2, init_filters * 4)

    middle_block = conv_stack(max_pool3, init_filters * 8)

    decoder1 = decoder_block(middle_block, conv3, init_filters * 4)
    decoder2 = decoder_block(decoder1, conv2, init_filters * 2)
    decoder3 = decoder_block(decoder2, conv1, init_filters)

    # soft = Softmax(axis=1)(decoder3)
    output_a = Conv2D(num_bins, 1, padding='same', activation='softmax')(decoder3)
    # output_b = Conv2D(32, 1, padding='same', activation='softmax')(decoder3)

    model = Model(inputs, output_a)
    return model

In [17]:
model_a = get_model(SIZE, 64)
model_b = get_model(SIZE, 64)
# model.summary()

In [18]:
# Load Model
model_a.load_weights("aero-color_a_10k.h5")
model_b.load_weights("aero-color_b_10k.h5")

In [19]:
# weight_a = [276.4182377999916, 10.096705660138996, 2.160336840606263, 1.8126515533081804, 0.3854523724941714,
#             0.22361807604925382, 1.040882132359878, 1.6375415188587383, 4.57322774863995, 37.81061231083443]

weight_a = [25.418237, 5.09670, 1.56033, 1.212651, 0.385452,
            0.2236180, 1.04088, 1.137541, 1.5732, 10.8106]

# weight_b = [263.46130653, 6.29587218, 2.67742605, 2.33413422, 0.63087839,
#             0.31916905, 0.66331561, 0.56931914, 1.04141953, 10.7601525]
weight_b = [23.46130, 2.29587, 1.67742, 1.23413, 0.63087,
            0.31916, 0.66331, 0.56931, 0.84141, 2.7601]
# model_a.compile(optimizer=Adam(0.01), loss=weighted_categorical_crossentropy(weights=weight_a), metrics='accuracy')
# model_b.compile(optimizer=Adam(0.01), loss=weighted_categorical_crossentropy(weights=weight_b), metrics='accuracy')


In [98]:
import os

# Make data generator
IMAGE_DIR = 'data/bnw/'
A_DIR = 'data/a/'
B_DIR = 'data/b/'

img_paths = sorted(
    [
        os.path.join(IMAGE_DIR, fname)
        for fname in os.listdir(IMAGE_DIR)
        if fname.endswith(".jpeg")
    ]
)
a_paths = sorted(
    [
        os.path.join(A_DIR, fname)
        for fname in os.listdir(A_DIR)
        if fname.endswith(".jpeg") and not fname.startswith(".")
    ]
)

b_paths = sorted(
    [
        os.path.join(B_DIR, fname)
        for fname in os.listdir(B_DIR)
        if fname.endswith(".jpeg") and not fname.startswith(".")
    ]
)


In [21]:
import random

total_samples = N

# random.Random(42).shuffle(img_paths)
# random.Random(42).shuffle(a_paths)
# random.Random(42).shuffle(b_paths)

img_paths = img_paths[:total_samples]
a_paths = a_paths[:total_samples]
b_paths = b_paths[:total_samples]

batch_size = 8

load_data = LoadData(
    batch_size, SIZE, img_paths, b_paths, bins
)

In [22]:
# Train Model
callbacks_a = [
    ModelCheckpoint("aero-color_a_ft.h5", save_best_only=True, save_weights_only=True, monitor='loss')
]

callbacks_b = [
    ModelCheckpoint("aero-color_b_ft.h5", save_best_only=True, save_weights_only=True, monitor='loss')
]

model_a.fit(load_data, epochs=epochs, verbose=1, batch_size=batch_size, callbacks=callbacks_a)
model_b.fit(load_data, epochs=epochs, verbose=1, batch_size=batch_size, callbacks=callbacks_b)

In [196]:
INDEX = 25000
X = cv2.imread(img_paths[INDEX], 0) / 255.0
a = cv2.imread(a_paths[INDEX], 0)
b = cv2.imread(b_paths[INDEX], 0)


a_dummy = np.digitize(a, bins) - 1
b_dummy = np.digitize(b, bins) - 1

# X = cv2.imread('data/bnw/8.jpeg', 0) / 255.0
# a = cv2.imread('data/a/8.jpeg', 0)
# b = cv2.imread('data/b/8.jpeg', 0)


In [197]:
a_hats = model_a.predict(X.reshape(1, SIZE, SIZE, 1))
b_hats = model_b.predict(X.reshape(1, SIZE, SIZE, 1))

In [199]:
# bin_vals_arr = np.array(bin_vals)

a_hats = np.log(a_hats) / 0.38
a_hats = np.exp(a_hats) / np.sum(np.exp(a_hats), axis=3, keepdims=True)
a_hats = np.matmul(a_hats, bin_val)

b_hats = np.log(b_hats) / 0.38
b_hats = np.exp(b_hats) / np.sum(np.exp(b_hats), axis=3, keepdims=True)
b_hats = np.matmul(b_hats, bin_val)


In [200]:
# a_hats = np.argmax(a_hats, axis=3)
# b_hats = np.argmax(b_hats, axis=3)

In [201]:
# Real Image
L = X.reshape((SIZE, SIZE)) * 255

lab = np.array([L, a, b]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('001.jpeg', img)
#

True

In [202]:
# Real Image Binned
# L = X.reshape((SIZE, SIZE)) * 255
# a_dummy = bins[a_dummy] + 0.5 * (bins[a_dummy + 1] - bins[a_dummy])
# b_dummy = bins[b_dummy] + 0.5 * (bins[b_dummy + 1] - bins[b_dummy])
# lab = np.array([L, a_dummy, b_dummy]).transpose((1, 2, 0)).astype('uint8')
# img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
# cv2.imwrite('002.jpeg', img)


True

In [204]:
L = X.reshape((SIZE, SIZE)) * 255
a_hat = a_hats[0, :, :].astype('int')
b_hat = b_hats[0, :, :].astype('int')
lab = np.array([L, a_hat, b_hat]).transpose((1, 2, 0)).astype('uint8')
# lab = np.array([L, a_hat, b]).transpose((1, 2, 0)).astype('uint8')
# lab = np.array([L, a, b_hat]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('002.jpeg', img)

True

In [205]:
# unique, counts = np.unique(a_binned, return_counts=True)
# counts = counts / counts.sum()
# # weights_a = counts.sum() / counts / num_bins
# print(dict(zip(unique, counts)))

In [206]:
cv2.imwrite('a_our.jpeg', a_hat)
cv2.imwrite('a.jpeg', a)
cv2.imwrite('b_our.jpeg', b_hat)
cv2.imwrite('b.jpeg', b)

True

In [207]:
cv2.imwrite('a_dummy.jpeg', a_dummy)
cv2.imwrite('b_dummy.jpeg', b_dummy)
cv2.imwrite('bnw.jpeg', L)

True

In [208]:
# unique, counts = np.unique(a_hats, return_counts=True)
# counts = counts / counts.sum()
# # weights_a = counts.sum() / counts / num_bins
# print(dict(zip(unique, counts)))
#
# unique, counts = np.unique(a_dummy, return_counts=True)
# counts = counts / counts.sum()
# # weights_a = counts.sum() / counts / num_bins
# print(dict(zip(unique, counts)))